In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from bs4 import BeautifulSoup
import lxml
import requests

#크롤링 하려는 정보 dict 처리
eminem_video_info = {'title':'', 'video_link':'', 'img_link':'', 'play_time':'',
'hits' : '', 'updated_time':'' }

In [14]:
def get_eminem_video_link(target_url):
    response = requests.get(target_url)
    soup = BeautifulSoup(response.text, "lxml")
    lis = soup.find_all('li', {'class' : 'channels-content-item yt-shelf-grid-item'})
    for li in lis :
        #<a class="yt-uix-sessionlink yt-uix-tile-link spf-link yt-ui-ellipsis yt-ui-ellipsis-2" dir="ltr" title="Eminem - Phenomenal (Behind The Scenes)" href="/watch?v=NEGjmd_RzLU">Eminem - Phenomenal (Behind The Scenes)</a>
        title = li.find('a', {'title' : True})['title']
        video_link = 'https://www.youtube.com' + li.find('a', {'href' : True})['href']
        img_link = li.find('img', {'src' : True})['src']
        #<span class="video-time" aria-hidden="true"><span aria-label="8분, 55초">8:55</span></span>
        play_time = li.find('span', {'class' : 'video-time'}).text
        #<ul class="yt-lockup-meta-info"><li>조회수 2,902,617회</li><li>6개월 전</li></ul>
        #hits_and_updated_time = li.find('ul', {'class' : 'yt-lockup-meta-info'})
        hits = li.find_all('li')[2].text
        updated_time = li.find_all('li')[3].text
        eminem_video_info = { 
            'title' : title, 'video_link' : video_link, 'img_link' : img_link, 'play_time' : play_time, 'hits' : hits, 'updated_time' : updated_time } 
        print(eminem_video_info)
    return eminem_video_info

        
target_url = 'https://www.youtube.com/user/LOENENT/videos'
get_eminem_video_link(target_url)

{'title': '[소란] ASMR LIVE 가수는 노래 제목 따라간다던데...', 'video_link': 'https://www.youtube.com/watch?v=-Ibg7BuVrUg', 'img_link': 'https://i.ytimg.com/vi/-Ibg7BuVrUg/hqdefault.jpg?sqp=-oaymwEiCMQBEG5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==&rs=AOn4CLAFg9eJcRwVLv0jYW2X6stdylYWKA', 'play_time': '1:01', 'hits': '조회수 45회', 'updated_time': '36분 전'}
{'title': '[ORORA] 리모델링2', 'video_link': 'https://www.youtube.com/watch?v=ps9VTyCFAKE', 'img_link': 'https://i.ytimg.com/vi/ps9VTyCFAKE/hqdefault.jpg?sqp=-oaymwEiCMQBEG5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==&rs=AOn4CLBryGfxLbXGHQxYPfBlq4B71QH9Yg', 'play_time': '1:01', 'hits': '조회수 11회', 'updated_time': '36분 전'}
{'title': "[빈센트] Live '잊혀지는게' (루프스테이션)", 'video_link': 'https://www.youtube.com/watch?v=GrQF4CfEMqI', 'img_link': 'https://i.ytimg.com/vi/GrQF4CfEMqI/hqdefault.jpg?sqp=-oaymwEiCMQBEG5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==&rs=AOn4CLBbbC3QlLd3PObchJm2ik5oiSNdSQ', 'play_time': '7:17', 'hits': '조회수 9회', 'updated_time': '36분 전'}
{'title': "[PS영준] 

{'title': "[우주히피] 'Daydream' (live clip)",
 'video_link': 'https://www.youtube.com/watch?v=r0Uw7nx5APA',
 'img_link': 'https://i.ytimg.com/vi/r0Uw7nx5APA/hqdefault.jpg?sqp=-oaymwEiCMQBEG5IWvKriqkDFQgBFQAAAAAYASUAAMhCPQCAokN4AQ==&rs=AOn4CLCrQ9NyzuYXBmBHEaR1qQZOngvhMg',
 'play_time': '2:35',
 'hits': '조회수 1,152회',
 'updated_time': '4일 전'}

# Youtube API 공통 코드

In [21]:
#-*-coding:utf-8
"""
Youtube data API 공통 코드 시작
원본: https://developers.google.com/youtube/v3/docs/commentThreads/list

soma0sd가 변경한 이 소스코드의 라이선스는 원본의 라이선스를 따릅니다.
"""
import httplib2
import os
import sys

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import argparser, run_flow

CLIENT_SECRETS_FILE = "client_secret.json"

YOUTUBE_READ_WRITE_SSL_SCOPE = "https://www.googleapis.com/auth/youtube.force-ssl"
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"

MISSING_CLIENT_SECRETS_MESSAGE = "WARNING: Please configure OAuth 2.0"


def get_authenticated_service(args):
    flow = flow_from_clientsecrets(
        CLIENT_SECRETS_FILE,
        scope=YOUTUBE_READ_WRITE_SSL_SCOPE,
        message=MISSING_CLIENT_SECRETS_MESSAGE)

    storage = Storage("youtube-api-snippets-oauth2.json")
    credentials = storage.get()

    if credentials is None or credentials.invalid:
        credentials = run_flow(flow, storage, args)
    return build(
        API_SERVICE_NAME,
        API_VERSION,
        http=credentials.authorize(httplib2.Http()))


args = argparser.parse_args()
service = get_authenticated_service(args)


def print_results(results):
    print(results)


def build_resource(properties):
    resource = {}
    for p in properties:
        prop_array = p.split('.')
        ref = resource
        for pa in range(0, len(prop_array)):
            is_array = False
            key = prop_array[pa]
            if key[-2:] == '[]':
                key = key[0:len(key) - 2:]
                is_array = True
            if pa == (len(prop_array) - 1):
                if properties[p]:
                    if is_array:
                        ref[key] = properties[p].split(',')
                    else:
                        ref[key] = properties[p]
            elif key not in ref:
                ref[key] = {}
                ref = ref[key]
            else:

                ref = ref[key]
    return resource


def remove_empty_kwargs(**kwargs):
    good_kwargs = {}
    if kwargs is not None:
        for key, value in kwargs.items():
            if value:
                good_kwargs[key] = value
    return good_kwargs
"""
Youtube data API 공통 코드 끝
이 아래에 사용할 기능을 코딩합니다.
"""

'\nYoutube data API 공통 코드 시작\n원본: https://developers.google.com/youtube/v3/docs/commentThreads/list\n\nsoma0sd가 변경한 이 소스코드의 라이선스는 원본의 라이선스를 따릅니다.\n'

usage: ipykernel_launcher.py [--auth_host_name AUTH_HOST_NAME]
                             [--noauth_local_webserver]
                             [--auth_host_port [AUTH_HOST_PORT [AUTH_HOST_PORT ...]]]
                             [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\jayjunglim\AppData\Roaming\jupyter\runtime\kernel-67430c57-2358-43be-8968-8e831a9a7223.json


SystemExit: 2

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
%tb

SystemExit: 2

In [18]:
import os
os.getcwd()

'C:\\Users\\jayjunglim\\Documents\\GitHub\\김용대_고급빅데이터분석'